<a href="https://colab.research.google.com/github/Yusadem131/Stage-D-Code/blob/main/Wrangling_time_series_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 



In [ ]:
df=pd.read_csv( "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv" ) 

In [ ]:
df[ "Month" ] = pd.to_datetime(df[ "Month" ]) #convert column to datetime 
df.set_index( "Month" , inplace= True ) 
plt.figure(figsize=( 10 , 6 )) 
plt.plot(df.index, df.Passengers, '--' , marker= '*' , ) 
plt.grid() 
plt.xlabel( 'Year' ) 
plt.ylabel( 'Passengers' ) 

In [ ]:
#check for missing values 
df.isnull().values.any() 


In [ ]:
#missing values, we will make a copy of our dataset and delete some values at random. 
df_copy = df.copy() 
rows = df_copy.sample(frac= 0.1 , random_state= 0 ) 
rows[ 'Passengers' ] = np.nan 
df_copy.loc[rows.index, 'Passengers' ] = rows[ 'Passengers' ] 
df_copy.isnull().sum()
#There are now 14 missing values in the dataset 

In [ ]:
#Filling missing data by imputation - Forward fill 
df_copy_ffill = df_copy.fillna(method= 'ffill' ) 
df_copy_ffill.isnull().sum() 

In [ ]:
#Filling missing data by imputation - Backward fill 
df_copy_bfill = df_copy.fillna(method= 'bfill' ) 
df_copy_bfill.isnull().sum()

In [ ]:
#Filling missing data by interpolation 
df_copy_LIF = df_copy.interpolate(method= 'linear' , limit_direction= 'forward' ) 
df_copy_LIF.isnull().sum() 
df_copy_LIB = df_copy.interpolate(method= 'linear' , limit_direction= 'backward' ) 
df_copy_LIB.isnull().sum() 

In [ ]:
#Downsampling and Upsampling 
#Downsample to quarterly data points 
df_quarterly = df.resample( '3M' ).mean() 


In [ ]:
#Upsample to daily data points 
df_daily = df.resample( 'D' ).mean() 

In [ ]:
df_MA = df.copy() 
MA = df_MA[ 'Passengers' ].rolling( 12 ).mean()

In [ ]:
#Time Series Specific Exploratory Methods 
import statsmodels.api as sm 
from pylab import rcParams 
rcParams[ 'figure.figsize' ] = 15 , 8 
decompose_series = sm.tsa.seasonal_decompose(df[ 'Passengers' ], model= 'additive' ) 
decompose_series.plot() 
plt.show() 
#The decomposed time series show an obvious increasing trend and seasonality variations. 
#Recall that we have initially plotted the moving average on the last 12 months which showed 
#that it varies with time. This suggests that the data is not stationary. We will now perform 
#an ADF test to confirm this speculation 

In [ ]:
from statsmodels.tsa.stattools import adfuller 
adf_result = adfuller(df['Passengers']) 
print(f'ADF Statistic: {adf_result[0]}')
print( f'p-value: {adf_result[1]}') 
print( f'No. of lags used: {adf_result[2]}') 
print( f'No. of observations used : {adf_result[3]}') 
print('Critical Values:') 
for k, v in adf_result[4].items(): 
 print( f' {k} : {v} ' ) 

 #From the results obtained, the p-value is greater than the critical value at a 5% 
#significance level and, the ADF statistic is greater that any of the critical values 
obtain. #This confirms that the series is indeed non-stationary. 